## **1. Install YOLOv8**

You just need Python ≥3.8 and pip.

```bash
pip install ultralytics
```

After installation, you can verify:

```bash
yolo
```

This should print YOLOv8’s CLI help menu.

---

## **2. Run a quick test**

Download a pretrained YOLOv8 model and run detection on an image:

```bash
yolo predict model=yolov8n.pt source='https://ultralytics.com/images/bus.jpg'
```

* `yolov8n.pt` → "nano" model (fastest, smallest).
* Other versions: `yolov8s.pt` (small), `yolov8m.pt` (medium), `yolov8l.pt` (large), `yolov8x.pt` (extra large).

Output will be in a `runs/predict` folder with bounding boxes drawn.

---

## **3. Train on your own dataset**

If you have a dataset:

```bash
yolo train model=yolov8n.pt data=data.yaml epochs=50 imgsz=640
```

* `data.yaml` should describe:

  ```yaml
  train: path/to/train/images
  val: path/to/val/images
  nc: 2   # number of classes
  names: ['cat', 'dog']
  ```

---

## **4. Use it from Python**

You can also run it as a Python module:

```python
from ultralytics import YOLO

# Load model
model = YOLO("yolov8n.pt")

# Predict on an image
results = model.predict("bus.jpg")
results[0].show()  # show image with boxes

# Train on custom data
model.train(data="data.yaml", epochs=50, imgsz=640)
```

---

## **5. Deployment options**

YOLOv8 supports exporting models to:

```bash
yolo export model=yolov8n.pt format=onnx
yolo export model=yolov8n.pt format=torchscript
yolo export model=yolov8n.pt format=engine  # TensorRT
```

That means you can deploy it on:

* **ONNX Runtime** (cross-platform)
* **TorchScript** (PyTorch native)
* **TensorRT** (NVIDIA GPUs, super fast)

---

## **6. License reminder**

YOLOv8 is under **AGPL-3.0**:

* ✅ Fine for research, personal projects, competitions (e.g., Kaggle).
* ⚠ If building a closed-source commercial service → either comply with AGPL (share source) or buy a commercial license from Ultralytics.

---



## **End-to-end YOLOv8 workflow**

### 1) Install

```bash
pip install ultralytics
yolo  # should print the CLI help
```

### 2) Dataset layout (YOLO format)

```
dataset/
├─ images/
│  ├─ train/   0001.jpg 0002.jpg ...
│  ├─ val/     1001.jpg 1002.jpg ...
│  └─ test/    2001.jpg 2002.jpg ...   # optional but recommended
└─ labels/
   ├─ train/   0001.txt 0002.txt ...
   ├─ val/     1001.txt 1002.txt ...
   └─ test/    2001.txt 2002.txt ...
```

Each `*.txt` file matches its image name and contains **one line per object**:

```
<class_id> <x_center> <y_center> <width> <height>
```

All values are **normalized to \[0,1]** relative to image width/height. Example (two objects):

```
0 0.512 0.433 0.120 0.210
3 0.245 0.680 0.050 0.090
```

> If an image has **no objects**, include an **empty** `*.txt` file (present, but 0 lines).

### 3) data.yaml

Create `data.yaml`:

```yaml
path: /absolute/or/relative/path/to/dataset
train: images/train
val: images/val
test: images/test          # optional

nc: 3
names: [cat, dog, rabbit]  # class names in order of IDs 0..nc-1
```

### 4) Quick sanity checks (strongly recommended)

```bash
# Verify labels and visualize a few samples
yolo data=data.yaml task=detect mode=check
```

This catches broken paths, missing files, or label format issues.

### 5) Train (from pretrained weights)

Pick a model size (`n/s/m/l/x` = nano→xlarge):

```bash
# Fast start
yolo train model=yolov8n.pt data=data.yaml epochs=100 imgsz=640 batch=16

# Helpful flags you may want:
# device=0               # pick a GPU; "0,1" for multi-GPU
# patience=30            # early stopping patience (epochs)
# cos_lr=True            # cosine LR schedule
# augment=True           # on by default for detect; can disable with augment=False
# project=./runs         # change output root
# name=my_yolov8_exp     # run name (folder)
```

Notes:

* **Anchor-free** head → no need to compute anchors.
* **Mixed precision** (AMP) is on by default when CUDA is available.
* If your images are large and you hit OOM: lower `imgsz` or `batch`.

### 6) Monitor & evaluate

Training automatically produces:

* **mAP\@0.5** and **mAP\@0.5:0.95** (the key metrics)
* Precision/Recall curves, confusion matrix, PR curves per class
* Best/last checkpoints: `runs/detect/train*/weights/best.pt`

Validate (e.g., after training or on a new split):

```bash
yolo val model=runs/detect/train/weights/best.pt data=data.yaml imgsz=640
```

### 7) Inference on images / folders / videos

```bash
# Single image or folder
yolo predict model=runs/detect/train/weights/best.pt source=path/to/images_or_video

# Save results in runs/predict/* with drawn boxes
```

### 8) Export for deployment

```bash
# Common targets
yolo export model=best.pt format=onnx
yolo export model=best.pt format=torchscript
yolo export model=best.pt format=engine  # TensorRT (NVIDIA GPUs)

# Then run with ONNX Runtime / TorchScript / TensorRT as needed
```

### 9) Using Python API (train + infer programmatically)

```python
from ultralytics import YOLO

# Load a pretrained detector
model = YOLO("yolov8n.pt")

# Train
model.train(data="data.yaml", epochs=100, imgsz=640, batch=16)

# Validate
metrics = model.val(data="data.yaml", imgsz=640)

# Predict
results = model.predict(source="path/to/img_or_dir")
for r in results:
    boxes = r.boxes.xyxy  # tensor[N, 4]
    scores = r.boxes.conf
    cls_ids = r.boxes.cls
```

### 10) Practical tips & gotchas

* **Labeling tools**: CVAT, Label Studio, Roboflow can export YOLO format directly.
* **Class imbalance**: consider more epochs, stronger augmentation, or focal loss (`--fl_gamma=2.0` via CLI args if exposed) — or oversample minority classes.
* **Augmentation**: YOLOv8 includes mosaic/mixup, HSV, flips by default; you can tune/disable in args.
* **Input channels**: models expect 3-channel images.

  * If you have **grayscale** (e.g., MRI), either save as 3-channel copies or adapt the dataloader (simple replication of the single channel to RGB works well).
* **Empty images**: keep empty label files so the trainer learns negatives.
* **Overfitting**: watch train vs val mAP; increase augmentation, add more data, or use a smaller model (`n`/`s`).
* **Resume training**: `yolo train resume=True` pointing to the last run directory.

### 11) Minimal checklist

* [ ] Images + YOLO TXT labels aligned by filename
* [ ] `data.yaml` with correct paths, `nc`, `names`
* [ ] Sanity check: `yolo ... mode=check`
* [ ] Train, then validate and inspect mAP/PR curves
* [ ] Export to the runtime you need (ONNX/TensorRT/etc.)

